In [0]:
# Model-1

In [0]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [0]:
df = pd.read_csv('https://zenodo.org/record/400614/files/apache.csv?download=1',sep=',')

In [0]:
df.head()

,id,product,component,reporter,bug_status,resolution,priority,bug_severity,version,short_desc,opendate,dup_list,root_id,disc_id
0,2,Log4j -,Layout,bugzilla,CLOS,FIXE,P3,nor,unspe,Just testing the Boogzeela setup for log4j,2001-01-08,NaN,NaN,NaN
1,3,Log4j -,Appender,matthew_scully,RESO,FIXE,P3,nor,unspe,file sharing,2001-01-08,NaN,NaN,NaN
2,22,Apache h,All,greenrd,CLOS,WONT,P5,enh,2.0-H,Need hooks for user-defined error handling,2001-01-09,NaN,NaN,NaN
3,27,Log4j -,Layout,richard.mccarthy,RESO,INVA,P1,nor,1.0,Error in the formatting of the (%F:%L) layout ...,2001-01-10,NaN,NaN,NaN
4,29,Log4j -,Other,woge,RESO,FIXE,P3,nor,1.0,"Calling Category.error(Object, Throwable) thro...",2001-01-11,NaN,NaN,NaN


**Remove NaN from the short_desc**

In [0]:
df.isnull().sum()

id                  0
product             0
component           0
reporter            0
bug_status          0
resolution          0
priority            0
bug_severity        0
version             0
short_desc         58
opendate            0
dup_list        41531
root_id         38016
disc_id         38016
dtype: int64

In [0]:
df = df[['id','short_desc','dup_list']]
df.head()

,id,short_desc,dup_list
0,2,Just testing the Boogzeela setup for log4j,NaN
1,3,file sharing,NaN
2,22,Need hooks for user-defined error handling,NaN
3,27,Error in the formatting of the (%F:%L) layout ...,NaN
4,29,"Calling Category.error(Object, Throwable) thro...",NaN


In [0]:
df.shape

(44049, 3)

Remove Blank Records

In [0]:
blanks = []

for i,id,sd,dl in df.itertuples():
    if type(sd)==str:
        if sd.isspace():
            blanks.append(i)
    else:
        blanks.append(i)
len(blanks)  

58

In [0]:
df.drop(blanks,inplace=True)
df.isnull().sum()

id                0
short_desc        0
dup_list      41474
dtype: int64

**Similarity Method:**

In [0]:
#conda install -c conda-forge spacy
#conda install -c conda-forge spacy-lookups-data
#!python -m spacy download en_core_web_lg

In [0]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [0]:
def to_string(df,id):
    s = df[df['id'] == id]
    s = s.short_desc.to_string(index=False)
    return s[1:]

In [0]:
s1 = to_string(df,136)
print(s1)
s2 = to_string(df,535)
print(s2)

Infinite loop in PrefixManager.java BugRat Rep...
deadlock in connection handler


**Remove Puntuations**

In [0]:
#Remove Puntuations
def remove_puntuation(str):
    import re
    return re.sub(r'[^\w\s]','',str)

In [0]:
s1 = remove_puntuation(s1)
s2 = remove_puntuation(s2)

**Remove Stop Words**

In [0]:
#Remove Stop Words
def remove_stop_word(sentense): # accept string
    s = []
    sentense = nlp(sentense)
    for token in sentense:
        if token.is_stop == False:
            s.append(token) 
    #Convert list to string
    str1 = ""    
    for ele in s:  
        str1 += str(ele) +' '     
    return nlp(str1[:-1]) #return doc 

In [0]:
doc1 = remove_stop_word(s1)
doc2 = remove_stop_word(s2)

In [0]:
print('Similarity: ' + str(doc1.similarity(doc2)))

Similarity: 0.42781221066187264


**Build a function of all previous work**

In [0]:
# Put all prevouse work together in a function
def prep(df,id):
    s = to_string(df,id)
    s = remove_puntuation(s)
    doc = remove_stop_word(s)
    return doc

In [0]:
# Prepare the vectors for each document
import time
start_time = time.time()
c = 0
prepared = []
l = len(df.index)
for i,id,sd,db in df.itertuples():
    c = c + 1
    doc = prep(df,id)
    prepared.append((id,doc))
    c = c + 1
    if c % 100 == 0:
        progress = c/l*100
        print('Progress: %s %%' % round(progress, 2), end="\r", flush=True)
    if c > 1000:
        print('\nDone')
        break
print("--- %s seconds ---" % (time.time() - start_time))

Progress: 2.27 %
Done

--- 8.36637806892395 seconds ---


In [0]:
c = 0
for i in prepared:
    print(i[0], '\t', i[1].text)
    c = c + 1
    if c > 5:
        break

In [0]:
# Calculate one by one similarity score
start_time = time.time()
similarities = []
l = len(prepared)
c = 0
for i in prepared:
    c = c + 1
    for j in prepared:
        score = i[1].similarity(j[1])
        similarities.append((i[0],j[0],score))
    if c % 10 == 0:
        progress = c/l*100
        print('Progress: %s %%' % round(progress, 2), end="\r", flush=True)
print('\nDone')
print("--- %s seconds ---" % (time.time() - start_time))

Progress: 99.8 %%
Done
--- 150.92698454856873 seconds ---


In [0]:
dfObj = pd.DataFrame(similarities) 

In [0]:
sorted_similarities = sorted(similarities, key=lambda tup: tup[2], reverse=True)

In [0]:
sorted_similarities[510:550]

[(560, 560, 1.0),
 (561, 561, 1.0),
 (562, 562, 1.0),
 (563, 563, 1.0),
 (564, 564, 1.0),
 (565, 565, 1.0),
 (566, 566, 1.0),
 (567, 567, 1.0),
 (569, 569, 1.0),
 (48, 54, 0.9999999801943156),
 (48, 441, 0.9999999801943156),
 (54, 48, 0.9999999801943156),
 (54, 441, 0.9999999801943156),
 (441, 48, 0.9999999801943156),
 (441, 54, 0.9999999801943156),
 (43, 48, 0.9999999741066152),
 (43, 54, 0.9999999741066152),
 (43, 441, 0.9999999741066152),
 (48, 43, 0.9999999741066152),
 (48, 373, 0.9999999741066152),
 (48, 553, 0.9999999741066152),
 (54, 43, 0.9999999741066152),
 (54, 373, 0.9999999741066152),
 (54, 553, 0.9999999741066152),
 (373, 48, 0.9999999741066152),
 (373, 54, 0.9999999741066152),
 (373, 441, 0.9999999741066152),
 (441, 43, 0.9999999741066152),
 (441, 373, 0.9999999741066152),
 (441, 553, 0.9999999741066152),
 (553, 48, 0.9999999741066152),
 (553, 54, 0.9999999741066152),
 (553, 441, 0.9999999741066152),
 (213, 559, 0.9999999672100989),
 (559, 213, 0.9999999672100989),
 (43, 

In [0]:
def test(df,id1,id2):
    s1 = to_string(df,id1)
    print('s1: ' + s1)
    s2 = to_string(df,id2)
    print('s2: ' + s2)
    s1 = remove_puntuation(s1)
    s2 = remove_puntuation(s2)
    doc1 = remove_stop_word(s1)
    print('Doc1: ' + doc1.text + '\t')
    doc2 = remove_stop_word(s2)
    print('Doc2: ' + doc2.text + '\t')
    print('Similarity: ' + str(doc1.similarity(doc2)))

In [0]:
test(df,48,441)

s1: hueRotate bug in ColorMatrix
s2: [BUG] BugRat Report#759
Doc1: hueRotate bug ColorMatrix	
Doc2: BUG BugRat Report759	
Similarity: 0.9999999801943156


In [0]:
dfObj.to_csv('similarities', index = False)